# Import Packages

In [73]:
%env MUJOCO_GL=egl
import time
from collections import deque

import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import nengo
import numpy as np
import PIL.Image

%matplotlib widget

# Access to enums and MuJoCo library functions.
from dm_control.mujoco.wrapper.mjbindings import enums
from dm_control.mujoco.wrapper.mjbindings import mjlib

from dm_control import composer, mjcf, mujoco
from dm_control.utils import inverse_kinematics as ik
from IPython.display import HTML
from multiprocessing import Process

env: MUJOCO_GL=egl


# Define Utilities and Parameters

In [55]:
# Rendering parameters
dpi = 100
framerate = 24 # (Hz)
width, height = 720, 480
sensor_shape = (10, 10)

# IK solver parameters
_MAX_STEPS = 50
_TOL = 1e-12


def display_video(frames, framerate=30):
    height, width, _ = frames[0].shape
    orig_backend = matplotlib.get_backend()
    matplotlib.use('Agg')  # Switch to headless 'Agg' to inhibit figure rendering.
    fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
    matplotlib.use(orig_backend)  # Switch back to the original backend.
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.set_position([0, 0, 1, 1])
    im = ax.imshow(frames[0])
    def update(frame):
      im.set_data(frame)
      return [im]
    interval = 1000/framerate
    anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                   interval=interval, blit=True, repeat=False)
    return HTML(anim.to_html5_video())


def get_jacobian(physics, site_name):
    phys = mujoco.Physics.from_model(physics.model)
    jac_pos = np.zeros((3, phys.model.nv))
    jac_rot = np.zeros((3, phys.model.nv))
    mjlib.mj_jacSite(
        phys.model.ptr,
        phys.data.ptr,
        jac_pos,
        jac_rot,
        phys.model.name2id(site_name, 'site'))
    
    return jac_pos, jac_rot

# Run Simulations

In [85]:
# Load scene
scene_xml = 'models/scene.xml'
physics = mujoco.Physics.from_xml_path(scene_xml)

# Define simulation variables
site_name = 'attachment_site'
control_site = physics.data.site(name=site_name)
target_site = physics.data.site(name='reach_site2')
joint_names = ['joint{}'.format(i+1) for i in range(7)]
ctrl = np.zeros(10)
duration = 1.0 # (seconds)
omega = np.pi/2 # Rotator angular velocity
video = []
sensor_data = []

# Simulate, saving video frames
physics.reset(0)
ctrl[:7] = physics.data.qpos[:7]
xpos = control_site.xpos
t0 = time.time()
while physics.data.time < duration:
    if control_site.xpos[2] > target_site.xpos[2]:
        xpos[2] -= 0.05
        result = ik.qpos_from_site_pose(
          physics=physics,
          site_name=site_name,
          target_pos=xpos,
          target_quat=[0, 0, 0, 1],
          joint_names=joint_names,
          tol=_TOL,
          max_steps=_MAX_STEPS,
          inplace=False,
        )
        if result.success:
            ctrl[:7] = result.qpos[:7]
    ctrl[-3:] = np.array([1, 0, 1])*omega*physics.data.time
    physics.set_control(ctrl)
    physics.step()

    # Save video frames and sensor data
    if len(video) < physics.data.time * framerate:
        pixels = physics.render(camera_id='prospective', width=width, height=height)
        video.append(pixels.copy())
        
        data = physics.data.sensordata
        print(data)
print("time elapsed: {}s".format(time.time() - t0))
# PIL.Image.fromarray(image)
display_video(video, framerate)

[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[363.78478473]
[224.45026263]
[0.]
[0.]
[0.]
[0.]
time elapsed: 44.557379722595215s


# Generate Strings to Define Sensor Sites and Touch Sensors to Inject into The XML file

In [82]:
n = 10
dx = 3e-3
offset = dx*(n-1)/2.0
for i in range(n*n):
    print('{}<site name="taxel_site{}" type="capsule" size=".001 .002" pos="{:.5f} -.0025 {:.5f}" euler="1.570796 0 0"/>'.format(' '*24, i+1, dx*(i//n)-offset, dx*(i%n)-offset))
for i in range(n*n):
    print('{}<touch name="taxel{}" site="taxel_site{}"/>'.format(' '*4, i+1, i+1))

                        <site name="taxel_site1" type="capsule" size=".001 .002" pos="-0.01350 -.0025 -0.01350" euler="1.570796 0 0"/>
                        <site name="taxel_site2" type="capsule" size=".001 .002" pos="-0.01350 -.0025 -0.01050" euler="1.570796 0 0"/>
                        <site name="taxel_site3" type="capsule" size=".001 .002" pos="-0.01350 -.0025 -0.00750" euler="1.570796 0 0"/>
                        <site name="taxel_site4" type="capsule" size=".001 .002" pos="-0.01350 -.0025 -0.00450" euler="1.570796 0 0"/>
                        <site name="taxel_site5" type="capsule" size=".001 .002" pos="-0.01350 -.0025 -0.00150" euler="1.570796 0 0"/>
                        <site name="taxel_site6" type="capsule" size=".001 .002" pos="-0.01350 -.0025 0.00150" euler="1.570796 0 0"/>
                        <site name="taxel_site7" type="capsule" size=".001 .002" pos="-0.01350 -.0025 0.00450" euler="1.570796 0 0"/>
                        <site name="taxel_site8" type="ca